In [2]:
import torch
import nltk
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from nltk.corpus import stopwords

# Load the pre-trained model and tokenizer
model_name = "mediabiasgroup/DA-RoBERTa-BABE"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [16]:


def biasScore(input_text):

    # Tokenize the input text
    input_tokens = tokenizer.tokenize(input_text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in input_tokens if token.lower() not in stop_words]
    filtered_text = tokenizer.convert_tokens_to_string(filtered_tokens)

    # Truncate the input to a maximum length of 512 tokens
    max_length = 512
    filtered_chunks = [filtered_text[i:i+max_length] for i in range(0, len(filtered_text), max_length)]

    # Get the predicted bias score for each chunk and average them
    bias_scores = []
    for chunk in filtered_chunks:
        inputs = tokenizer(chunk, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            bias_score = logits.softmax(dim=1)[0][1].item()
            bias_scores.append(bias_score)

    avg_bias_score = sum(bias_scores) / len(bias_scores)

    return avg_bias_score

In [12]:
import pandas as pd

fake = pd.read_csv('data/huggingface/Fake.csv')
true = pd.read_csv('data/huggingface/True.csv')

In [13]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [14]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [17]:
for i in range(20):
    print(biasScore(fake.loc[i]['text']))

0.7507886290550232
0.5814711637794971
0.7020251048462731
0.6410137265920639
0.23483406007289886
0.5802013427019119
0.8002024829387665
0.6451395392417908
0.768903930981954
0.8912675529718399
0.7859668731689453
0.7324478700757027
0.317046320438385
0.6588901244103909
0.6299581726392111
0.7963041841983796
0.7831303477287292
0.5740612000226974
0.8074799055854479
0.6978281587362289


In [18]:
for i in range(20):
    print(biasScore(true.loc[i]['text']))

0.07637494914233685
0.058860898949205875
0.12793080819149813
0.08153676465153695
0.0865131721577861
0.07283773832023144
0.10381728410720825
0.8717984557151794
0.18765614181756973
0.03399055451154709
0.04498431161046028
0.0693562114611268
0.3467649221420288
0.12349899858236313
0.08431637138128281
0.04500198798875014
0.04746564891603258
0.4255669017632802
0.035699835419654845
0.042316327492396034
